# Proyecto 14: SQL 

## Introducción y objetivo
En este proyecto se explora una base de datos de servicios de libros electrónicos. 
El objetivo es utilizar los datos para obtener información relevante sobre los libros comercializados, para que una startup pueda generar propuesatas de valor en el desarrollo de nuevos productos. 

Los datos contenidos en la base de datos son los siguientes: 

books 

    - book_id: identificación del libro 
    - author_id: identificación del autor o autora 
    - title: título 
    - num_page: número de páginas 
    - publication_date: fecha de lanzamiento 
    - publisher_id: identificación de la editorial 

authors 

    - author_id: identificación del autor o autora 
    - author: Nombre del autor o autora 

publishers 

    - publisher_id: identificación de la editorial 
    - publisher:  nombre de la editorial 

ratings 

    - rating_id: identificación de la calificación 
    - book_id: identificación del libro 
    - username: Nombre de la persona que revisó el libro
    - rating: calificación 

reviews 

    - review_id: identificación de la reseña 
    - book_id: identificación del libro 
    - username: nombre del usuarió que revisó el libro 
    - text: texto de la reseña 

## Conexión a la base de datos 

In [1]:
# importar librerías
import pandas as pd
from sqlalchemy import create_engine

#conexión a base de datos
db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', #host de conexión
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos
#Aplicar string de conexió a postgresql
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])
# solicitando datos sin procesar
engine = create_engine(connection_string, connect_args={'sslmode':'require'})



## Estudio de las tablas 
Se imprimen las primeras filas de cada tabla para analizar los datos

In [2]:
#Definir query para SQL para books
query_books = '''
         SELECT *
         FROM books
         LIMIT 5
        '''
#Definir query para SQL para authors
query_authors = '''
         SELECT *
         FROM authors
         LIMIT 5
        '''
#Definir query para SQL para publishers
query_publishers = '''
         SELECT *
         FROM publishers
         LIMIT 5
        '''
#Definir query para SQL para ratings
query_ratings = '''
         SELECT *
         FROM ratings
         LIMIT 5
        '''
         
#Definir query para SQL para reviews
query_reviews = '''
         SELECT *
         FROM reviews
         LIMIT 5
         '''

In [3]:
#Mostar tabla con método read sql de pandas 
books = pd.io.sql.read_sql(query_books, con = engine)
books.head(5)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
#Mostar tabla con método read sql de pandas 
authors = pd.io.sql.read_sql(query_authors, con = engine)
authors

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
#Mostar tabla con método read sql de pandas 
publishers = pd.io.sql.read_sql(query_publishers, con = engine)
publishers

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
#Mostar tabla con método read sql de pandas 
ratings = pd.io.sql.read_sql(query_ratings, con = engine)
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
#Mostar tabla con método read sql de pandas 
reviews = pd.io.sql.read_sql(query_reviews, con = engine)
reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Hallazgos sobre el estudio de las tablas

## Búsqueda de número de libros publicados después del 1 de enero del 2001 

In [8]:
#Definir query para SQL para libros después de enero 2001
query_enero_2001 ='''

SELECT COUNT(*) as count_books -- Contar el número de libros
        FROM books
        WHERE publication_date > '2000-01-01'; -- Filtrar fecha
        
         '''
#Mostar tabla con método read sql de pandas
enero_2001 = pd.io.sql.read_sql(query_enero_2001, con = engine)
enero_2001

,count_books
0,819


### Hallazgos sobre consulta de libros

## Búsqueda de número de reseñas y calificaciones
Número de reseñas de usuarios y la calificación promedio de cada libro 

In [9]:
#Definir query para SQL para Número de reseñas de usuarios y la calificación promedio de cada libro
query_calificaciones ='''

SELECT title,
       COUNT(DISTINCT review_id) AS review_count, -- Selecciona valores únicos de reseñas de usaurio en caso de valores duplicados
       AVG(rating) AS rating -- Selecciona valores promedio de rating
        FROM books
         
       LEFT JOIN ratings ON ratings.book_id = books.book_id -- Unimos tabla ratings con books mediante book_id para obtener rating de cada libro
       LEFT JOIN reviews ON reviews.book_id = books.book_id -- Unimos tabla reviews con books mediante book_id para obtener reviews de cada libro
       GROUP BY books.book_id -- Agrupamos por libros
       ORDER BY review_count DESC; -- Ordenamos por cantida de reviews
       
         '''
#Mostar tabla con método read sql de pandas
calificaciones = pd.io.sql.read_sql(query_calificaciones, con = engine)
calificaciones.head(20) 

,title,review_count,rating
0,Twilight (Twilight #1),7,3.662500
1,Water for Elephants,6,3.977273
2,The Glass Castle,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,The Curious Incident of the Dog in the Night-Time,6,4.081081
5,The Da Vinci Code (Robert Langdon #2),6,3.830508
6,The Alchemist,6,3.789474
7,The Hobbit or There and Back Again,6,4.125000
8,The Book Thief,6,4.264151
9,The Lightning Thief (Percy Jackson and the Oly...,6,4.080645


In [10]:
#Repetimos query ahora ordenando por rating para concer libros con mayor calificación promedio
query_calificaciones2 ='''

SELECT title,
       COUNT(DISTINCT review_id) AS review_count, -- Selecciona valores únicos de reseñas de usaurio en caso de valores duplicados
       AVG(rating) AS rating -- Selecciona valores promedio de rating
       FROM books
         
       LEFT JOIN ratings ON ratings.book_id = books.book_id -- Unimos tabla ratings con books mediante book_id para obtener rating de cada libro
       LEFT JOIN reviews ON reviews.book_id = books.book_id -- Unimos tabla reviews con books mediante book_id para obtener reviews de cada libro
       GROUP BY books.book_id -- Agrupamos por libros
       ORDER BY rating DESC; -- Ordenamos por calificación promedio
       
         '''
#Mostar tabla con método read sql de pandas
calificaciones2 = pd.io.sql.read_sql(query_calificaciones2, con = engine)
calificaciones2.head(20) 

,title,review_count,rating
0,Arrows of the Queen (Heralds of Valdemar #1),2,5.0
1,The Walking Dead Book One (The Walking Dead #...,2,5.0
2,Light in August,2,5.0
3,Wherever You Go There You Are: Mindfulness Me...,2,5.0
4,Captivating: Unveiling the Mystery of a Woman'...,2,5.0
5,Tai-Pan (Asian Saga #2),2,5.0
6,The Adventures of Tom Sawyer and Adventures of...,1,5.0
7,Hard Times,2,5.0
8,A Fistful of Charms (The Hollows #4),2,5.0
9,In the Hand of the Goddess (Song of the Liones...,2,5.0


### Hallazgos sobre consulta de calificaciones
Observamos que los libros con más reseñas tienen calificación media es decir, entre 3 y 4 siendo que ninguno pasa de 4.5

Ahora bien, los libros con reseñas de excelente (5), salvo pocas excepciones tienen 2 reseñas, sería interesante revisar mediante un análisis exploratorio si esta tendencia se mantiene a lo largo de todo el dataset para saber si entre más reseñas, menor es la calificación y si estas estás siendo afectadas por valores atípicos como por ejemplo que la mayoría de usuarios marque algún título con 4, pero que un usuario qeu maruqe 1 haga bajar la calificación a 3. 

## Búsqueda de editoriales 
identificar la editorial que ha publicado el mayor número de libros con más de 50 páginas 
Esto último con el fin de incluir solo libros y excluir folletos y publicaciones similares 

In [11]:
#Definir query para SQL para Número de reseñas de usuarios y la calificación promedio de cada libro
query_editoriales=  '''

SELECT publisher,
     COUNT(DISTINCT book_id) AS book_count -- Selecciona valores únicos de libros en caso de valores duplicados
     FROM books
         
     LEFT JOIN publishers ON publishers.publisher_id = books.publisher_id -- Unimos tabla publishers con books mediante publisher_id para obtener nombre de la editorial de cada libro
     WHERE num_pages > 50 -- Filtramos libros con más de 50 páginas para excluir folletos y publicaciones similares 
     GROUP BY publisher -- Agrupamos por nombre de editorial 
     ORDER BY book_count DESC -- Ordenamos por cantidad de libros publicados 
     
        '''
#Mostar tabla con método read sql de pandas
editoriales = pd.io.sql.read_sql(query_editoriales, con = engine)
editoriales.head(5) 

,publisher,book_count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


In [12]:
editoriales.tail(5) 

,publisher,book_count
329,Random House Anchor,1
330,Random House Audio Publishing Group,1
331,Random House Large Print Publishing,1
332,Random House Trade,1
333,Random House Vintage,1


### Hallazgos sobre consulta de editoriales
Observamos que la editorial con mayr cantidad de libros es Penguin Books, la caul al parecer tiene uan gran variedad de libros (no sólo de un género ), y la editorial con menor cantidad es Random House, que como podemso observar, tiene varias susb editoriales como audio, vintage e impresiones largas.

## Búsqueda de autores 
Identificar al autor con la calificación más alta promedio del libro. 
Se debe revisar solo libros con al menos 50 califiaciones 

In [13]:
#Definir query para SQL para autor con la calificación más alta promedio
query_autor = ''' 

SELECT 
    author as author,
    AVG(rating) as avg_rating -- Selecionamos función AVG para obtener promedios
    FROM authors

    INNER JOIN books ON books.author_id = authors.author_id --Unimos books con authors mediante author_id para obtner nomnre del autor
    INNER JOIN ratings ON ratings.book_id = books.book_id --Unimos books con ratings mediante book_id para obtner rating del libro
    GROUP BY author -- Agrupamos por autor
    HAVING COUNT(rating) > 50 -- Filtramos autores con rating mayor a 5          
    ORDER BY avg_rating DESC; --Ordenamos por calificación promedio
    
    '''

#Mostar tabla con método read sql de pandas
autor= pd.io.sql.read_sql(query_autor, con = engine)
autor.head(5) 

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.288462
1,Agatha Christie,4.283019
2,Markus Zusak/Cao Xuân Việt Khương,4.264151
3,J.R.R. Tolkien,4.240964
4,Roald Dahl/Quentin Blake,4.209677


In [14]:
autor.tail(5) 

,author,avg_rating
18,Dan Brown,3.741259
19,Lois Lowry,3.738462
20,George Orwell/Boris Grabnar/Peter Škerl,3.729730
21,Stephenie Meyer,3.662500
22,John Steinbeck,3.643836


### Hallazgos sobre consulta de autores
Hay un total de 23 autores, siendo la que mejr reseñas tiene por libro J.K. Rowling seguida de Agatha Christie amabs con practicamente la misma calificación 4.28

Este dato resulta interesante ya que el libro Harry Potter and the Prisoner of Azkaban de la autora J.K. Rowling, se posiciona en los libros con mayor cantidad de reseñas, por lo que podemos decir que es un libro considerado bueno por la comunidad, ya que tiene buena reseñas a pesar de la cantidad de críticas ya que muchos libros con más críticas tienden a bajar calificación , pues como vemos en otras consultas los libros con calificación de 5 tienen alrededor de 2 reseñas.

## Búsqueda de reseñas 
Número promedio de reseñas de texto entre los usuarios de calificaron más de 50 libros

In [15]:
#Definir query para SQL para autor con la calificación más alta promedio
query_reseñas =''' 
SELECT AVG(COUNT) AS avg_n_reviews -- Seleccionamos el número de reseñas mediante COUNT y obtenemso promedio mediante AVG
    FROM
        (SELECT COUNT(text) -- Seleccionamos la cantidad de reseñas de texto de tabla reviews
            FROM reviews
            LEFT JOIN -- Unimos con tabla ratings (calificaciones) y reviews (reseñas de texto) medinte username
                (SELECT username, -- Usamos subquery para seleccioanr nombres de usuario de ratings
                COUNT(rating_id) rating_count --Contamos id de ratings
                FROM ratings 
                GROUP BY username) -- Agruapamos por nombre de usuario
                AS rating -- Damos valor de ratings
                ON rating.username = reviews.username --Unimos ambas tablas mediante username
                WHERE rating_count > 50 --Filtramos rating mayor a 50 
                GROUP BY reviews.username) AS COUNT; 
        '''
#Mostar tabla con método read sql de pandas
reseñas= pd.io.sql.read_sql(query_reseñas, con = engine)
reseñas

,avg_n_reviews
0,24.333333


### Hallazgos sobre consulta de reseñas
Observamos que el número promedio de reseñas de usuarios que calificaron más de 50 libros es de 24 

## Conclusiones
En este proyecto se utilzó SQL para manejar los datos y realizar los filtros requeridos, se tuvieron varios hallazgos siendo que se tiene un total de 23 autores con rating mayor a 5, siendo la autora con mayor rating promedio JK Rowling. 
Otro punto a destacar es que los usuarios más activos en el área de reseñas tiene un total de 24 reseñas en promedio, por lo que sería interesante revisar en futuros análisis que tipo de libros son los que reseñan estos usuarios pues si están activos en el área de reseñas significa que son clientes que adquieren mayor cantidad de libros. 
Por otro lado tenemos que la editorial con mayor cantidad de publicaciones es Penguin Books, por lo que podría ser interesante para una startup generar alianzas con dicha empresa debido al extenso catálogo, ahora bien se recomendaría también analizar que editoriales tienen los libros mejor calificados en futuros análsis. 
Finalmente al revisar reseñas y calificaciones vemos que la cantidad de reseñas puede afectar el rating más general, pues los libros con mayor cantidad de reseñas no llegan a los 5 puntos, siendo que los libros con calificaición máxima (5) no pasan de 2 reseñas. 
Así mismo se recomienda realizar análsis futuros para revisar si estos valores son atípicos como promedio de 5 y 1 que bajen calificación a 3, así como análsis de reseñas a lo largo del tiempo de ser posible, para confirmar si las reseñas bajan o suben a lo largo del tiempo lo que permitiría a una empresa esablecer tendencias según épocas del año o revisar tiempo de vida de lso libros según su popularidad.
